In [1]:
import pandas as pd

### 1. Clean Real Demand Data

In [37]:
# Load the files
real = pd.read_csv("./RealDemand_Sept2025.csv", sep=';')
real.head()

,id,name,geoid,geoname,value,datetime
0,1293,Real demand,NaN,NaN,23820.000,2025-09-01T00:00:00+02:00
1,1293,Real demand,NaN,NaN,22533.583,2025-09-01T01:00:00+02:00
2,1293,Real demand,NaN,NaN,21515.000,2025-09-01T02:00:00+02:00
3,1293,Real demand,NaN,NaN,20827.833,2025-09-01T03:00:00+02:00
4,1293,Real demand,NaN,NaN,20696.667,2025-09-01T04:00:00+02:00


In [38]:
# Extract datetime and value
real["datetime"] = pd.to_datetime(real["datetime"])
real = real[["datetime", "value"]].rename(columns={"value": "NLD"})

# Format to match demand.csv
# First two header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["demand_power"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": real["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": real["NLD"]
})

# Combine them together
real_demand = pd.concat([header, ts], ignore_index=True)

# Save to new CSV
real_demand.to_csv("./demand_sept2025.csv", index=False)

real_demand.head(10)


,nodes,NLD
0,techs,demand_power
1,2025-09-01 00:00:00,23820.0
2,2025-09-01 01:00:00,22533.583
3,2025-09-01 02:00:00,21515.0
4,2025-09-01 03:00:00,20827.833
5,2025-09-01 04:00:00,20696.667
6,2025-09-01 05:00:00,21107.833
7,2025-09-01 06:00:00,23696.917
8,2025-09-01 07:00:00,26555.25
9,2025-09-01 08:00:00,27857.25


### 2. Clean Renewables Data

In [50]:
# load solar data
solar = pd.read_csv("./ForecastGenerationSolar_Sept2025.csv", sep=';')
solar.tail()

,id,name,geoid,geoname,value,datetime
714,10034,Forecast generation Solar,NaN,NaN,3686.00,2025-09-30T19:00:00+02:00
715,10034,Forecast generation Solar,NaN,NaN,357.00,2025-09-30T20:00:00+02:00
716,10034,Forecast generation Solar,NaN,NaN,238.25,2025-09-30T21:00:00+02:00
717,10034,Forecast generation Solar,NaN,NaN,315.75,2025-09-30T22:00:00+02:00
718,10034,Forecast generation Solar,NaN,NaN,291.75,2025-09-30T23:00:00+02:00


In [51]:
# load wind data
wind = pd.read_csv("./MeasuredOnshoreWindGeneration_Sept2025.csv", sep=';')
wind.tail()

,id,name,geoid,geoname,value,datetime
715,1159,Measured Onshore wind generation,NaN,NaN,2857.048,2025-09-30T19:00:00+02:00
716,1159,Measured Onshore wind generation,NaN,NaN,3235.416,2025-09-30T20:00:00+02:00
717,1159,Measured Onshore wind generation,NaN,NaN,4015.845,2025-09-30T21:00:00+02:00
718,1159,Measured Onshore wind generation,NaN,NaN,4610.186,2025-09-30T22:00:00+02:00
719,1159,Measured Onshore wind generation,NaN,NaN,4986.580,2025-09-30T23:00:00+02:00


In [52]:
# add one column 'installed_capacity' with value 38659.3
solar["installed_capacity"] = 38659.3

# add one column 'CF_solar' as capacity factor, calculated as value / installed_capacity
solar["CF_solar"] = solar["value"] / solar["installed_capacity"]
solar.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_solar
0,10034,Forecast generation Solar,NaN,NaN,393.75,2025-09-01T00:00:00+02:00,38659.3,0.010185
1,10034,Forecast generation Solar,NaN,NaN,155.50,2025-09-01T01:00:00+02:00,38659.3,0.004022
2,10034,Forecast generation Solar,NaN,NaN,166.25,2025-09-01T02:00:00+02:00,38659.3,0.004300
3,10034,Forecast generation Solar,NaN,NaN,157.00,2025-09-01T03:00:00+02:00,38659.3,0.004061
4,10034,Forecast generation Solar,NaN,NaN,152.25,2025-09-01T04:00:00+02:00,38659.3,0.003938


In [53]:
# add one column 'installed_capacity' with value 33010.6
wind["installed_capacity"] = 33010.6

# add one column 'CF_wind' as capacity factor, calculated as value / installed_capacity
wind["CF_wind"] = wind["value"] / wind["installed_capacity"]
wind.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_wind
0,1159,Measured Onshore wind generation,NaN,NaN,11362.571,2025-09-01T00:00:00+02:00,33010.6,0.344210
1,1159,Measured Onshore wind generation,NaN,NaN,10791.856,2025-09-01T01:00:00+02:00,33010.6,0.326921
2,1159,Measured Onshore wind generation,NaN,NaN,10217.945,2025-09-01T02:00:00+02:00,33010.6,0.309535
3,1159,Measured Onshore wind generation,NaN,NaN,9947.512,2025-09-01T03:00:00+02:00,33010.6,0.301343
4,1159,Measured Onshore wind generation,NaN,NaN,9868.246,2025-09-01T04:00:00+02:00,33010.6,0.298942


In [56]:
# create final dataframe with datetime, CF_solar, CF_wind
renewables_df = pd.DataFrame({
    "datetime": solar["datetime"],
    "CF_solar": solar["CF_solar"],
    "CF_wind": wind["CF_wind"]
})

renewables_df.head()

,datetime,CF_solar,CF_wind
0,2025-09-01T00:00:00+02:00,0.010185,0.344210
1,2025-09-01T01:00:00+02:00,0.004022,0.326921
2,2025-09-01T02:00:00+02:00,0.004300,0.309535
3,2025-09-01T03:00:00+02:00,0.004061,0.301343
4,2025-09-01T04:00:00+02:00,0.003938,0.298942


In [57]:
# Extract datetime and value
renewables_df["datetime"] = pd.to_datetime(renewables_df["datetime"])
renewables_df = renewables_df[["datetime", "CF_solar", "CF_wind"]].rename(columns={"CF_solar": "NLD", "CF_wind": "NLD.1"})

# Format to match demand.csv
# First header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["solar_pv"], "NLD.1": ["wind_onshore"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": renewables_df["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": renewables_df["NLD"],
    "NLD.1": renewables_df["NLD.1"]
})

# Combine them together
renewables_final = pd.concat([header, ts], ignore_index=True)

# Save to new CSV
renewables_final.to_csv("./renewables_sept2025.csv", index=False)

renewables_final.head(10)

,nodes,NLD,NLD.1
0,techs,solar_pv,wind_onshore
1,2025-09-01 00:00:00,0.010185,0.34421
2,2025-09-01 01:00:00,0.004022,0.326921
3,2025-09-01 02:00:00,0.0043,0.309535
4,2025-09-01 03:00:00,0.004061,0.301343
5,2025-09-01 04:00:00,0.003938,0.298942
6,2025-09-01 05:00:00,0.001449,0.294695
7,2025-09-01 06:00:00,0.001345,0.294978
8,2025-09-01 07:00:00,0.002684,0.278224
9,2025-09-01 08:00:00,0.092985,0.252151
